# Изучение  процесса подбора гиперпараметров модели. 

## Подготовим среду и данные для выполнения заданий

### Загружаем датасет с информацией о медицинских параметрах больных диабетом

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabh00007/diabetescsv")

print("Path to dataset files:", path)

Path to dataset files: /home/sasha/.cache/kagglehub/datasets/saurabh00007/diabetescsv/versions/1


In [2]:
!ls $path

diabetes.csv


In [3]:
import pandas as pd
df = pd.read_csv(path + '/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
target = df['Outcome']
print(f'Размеры датасета: {df.shape}')

Размеры датасета: (768, 9)


In [5]:
from sklearn.model_selection import train_test_split

X = df.drop('Outcome', axis=1)

# Разделяем на train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, target, test_size=0.2, random_state=42, stratify=target  # сохраняем пропорции классов
)

## 1) Процесс подбора гиперпараметров используя Random Search 

In [6]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import time

# Параметры для подбора
param_dist = {
    "n_estimators": [50, 100, 150, 200],
    "max_depth": [None, 2, 4, 6, 8, 10],
    "max_features": ["sqrt", "log2", None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

# Создаём модель
rf = RandomForestClassifier(random_state=42)

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,            # число случайных комбинаций
    scoring='accuracy',   # метрика
    cv=5,
    random_state=42,
    n_jobs=-1
)

### Обучаем модель с лучшими параметрами

In [7]:
start = time.time()
random_search.fit(X_train, y_train)
end = time.time()

print("Random Search best params:", random_search.best_params_)
print("Random Search best score:", random_search.best_score_)
print("Time taken (s):", end - start)

y_pred = random_search.best_estimator_.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Random Search best params: {'n_estimators': 150, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 6}
Random Search best score: 0.7769425563108089
Time taken (s): 7.36449670791626
Test Accuracy: 0.7597402597402597


## 2) TPE Search (Hyperopt)

In [ ]:
!pip install hyperopt

In [11]:
import warnings
warnings.filterwarnings('ignore')

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score

n_estimators_list = [50, 100, 150, 200]
max_depth_list = [None, 2, 4, 6, 8, 10]
max_features_list = ["sqrt", "log2", None]
min_samples_split_list = [2, 5, 10]
min_samples_leaf_list = [1, 2, 4]

# Пространство гиперпараметров
space = {
    "n_estimators": hp.choice("n_estimators", n_estimators_list),
    "max_depth": hp.choice("max_depth", max_depth_list),
    "max_features": hp.choice("max_features", max_features_list),
    "min_samples_split": hp.choice("min_samples_split", min_samples_split_list),
    "min_samples_leaf": hp.choice("min_samples_leaf", min_samples_leaf_list)
}

# Функция для минимизации (негативная точность, т.к. fmin минимизирует)
def objective(params):
    clf = RandomForestClassifier(random_state=42, **params)
    score = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy').mean()
    return {'loss': -score, 'status': STATUS_OK}

trials = Trials()
start = time.time()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials)
end = time.time()

print("Hyperopt best params:", best)
print("Time taken (s):", end - start)

# Преобразуем индексы в значения параметров
best_params = {
    "n_estimators": n_estimators_list[best['n_estimators']],
    "max_depth": max_depth_list[best['max_depth']],
    "max_features": max_features_list[best['max_features']],
    "min_samples_split": min_samples_split_list[best['min_samples_split']],
    "min_samples_leaf": min_samples_leaf_list[best['min_samples_leaf']]
}

print("Hyperopt converted best params:", best_params)

100%|██████████████████████████████████████████████████████████████████████████| 20/20 [00:18<00:00,  1.08trial/s, best loss: -0.7834199653471945]
Hyperopt best params: {'max_depth': np.int64(0), 'max_features': np.int64(0), 'min_samples_leaf': np.int64(2), 'min_samples_split': np.int64(2), 'n_estimators': np.int64(3)}
Time taken (s): 18.49959087371826
Hyperopt converted best params: {'n_estimators': 200, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 10, 'min_samples_leaf': 4}


### Обучаем модель с лучшими параметрами

In [12]:
best_rf = RandomForestClassifier(random_state=42, **best_params)
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
print("Test Accuracy Hyperopt:", accuracy_score(y_test, y_pred))

Test Accuracy Hyperopt: 0.7532467532467533


## 3) Проанализируем полученные результаты
Несмотря на то, что Hyperopt на этапе кросс-валидации показал чуть более высокий score, реальная точность на тестовой выборке оказалась ниже, чем у Random Search. Это может быть связано с ограниченным числом итераций и спецификой распределения гиперпараметров, что привело к переобучению на кросс-валидации.

## Вывод: 
В данном случае Random Search оказался более эффективным, обеспечив лучшее качество на тестовых данных и быстрее завершив оптимизацию.